In [1]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

#Xgboost
#random forest
#Reseau de neurone
#Linear regression*

data = pd.read_csv('../data/processed/clean_dataset_.csv')
components = pd.read_csv('../data/processed/principalComponents_.csv')

X = components #composantes principales obtenu après l'ACP
y = data['Fire Alarm'] #variable cible

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#fonction qui permet d'évaluer chaque modèle
def evaluer_modele(y_test, y_pred, y_pred_proba):
    """
    Calcule et affiche les principales métriques d'évaluation d'un modèle de classification.
    Args:
        y_test (array-like): Les vraies étiquettes.
        y_pred (array-like): Les étiquettes prédites par le modèle.
        y_pred_proba (array-like): Les probabilités prédites pour la classe positive.
    """

    # Calcul des métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Création du DataFrame pour afficher les résultats
    data = {'Métrique': ['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC', 'Matrice de confusion'],
            'Valeur': [accuracy, precision, recall, f1, roc_auc, conf_matrix]}
    df = pd.DataFrame(data)

    print(df)




/usr/lib/python3/dist-packages/pytz/__init__.py:30: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


In [4]:
#---------------------------------   Random Forest  -------------------------------------------------------
# Créer un modèle de forêt aléatoire
model1 = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
model1.fit(X_train, y_train)

# Faire des prédictions   
y_pred1 = model1.predict(X_test)

# Obtenir les probabilités prédites
y_pred_proba1 = model1.predict_proba(X_test)[:, 1]
joblib.dump(model1, '../models/randomForest.joblib') 


evaluer_modele(y_test, y_pred1, y_pred_proba1)


               Métrique                  Valeur
0              Accuracy                     1.0
1             Precision                     1.0
2                Recall                     1.0
3              F1-score                     1.0
4                   AUC                     1.0
5  Matrice de confusion  [[2128, 0], [0, 5856]]


In [5]:
#Réseau de neurone




In [5]:
#XGBoost

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle XGBoost
model2 = XGBClassifier()

# Entraîner le modèle
model2.fit(X_train, y_train)

# Faire des prédictions
y_pred2 = model2.predict(X_test)

# Obtenir les probabilités prédites
y_pred_proba2 = model2.predict_proba(X_test)[:, 1] 

evaluer_modele(y_test, y_pred2, y_pred_proba2)
joblib.dump(model2, '../models/xgboost.joblib')


               Métrique                  Valeur
0              Accuracy                     1.0
1             Precision                     1.0
2                Recall                     1.0
3              F1-score                     1.0
4                   AUC                     1.0
5  Matrice de confusion  [[2128, 0], [0, 5856]]


['../models/xgboost.joblib']

In [6]:
#Regression Linéaire

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer un modèle de régression linéaire
model3 = LinearRegression()

# Entraîner le modèle
model3.fit(X_train, y_train)

# Faire des prédictions
y_pred3 = model3.predict(X_test)


# Évaluer le modèle
mse = mean_squared_error(y_test, y_pred3)
print("Mean Squared Error:", mse)
joblib.dump(model3, '../models/regressionLineaire.joblib')


Mean Squared Error: 0.04054753650764928


['../models/regressionLineaire.joblib']